In [3]:
#XXX pip install gffutils
from collections import defaultdict

import gffutils
import sqlite3

In [4]:
!rm -rf gambiae.g?f.gz ag.db 2>/dev/null
#!wget http://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gtfgz -O gambiae.gtf.gz
!wget http://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz -O gambiae.gff.gz

--2018-05-04 12:02:39--  http://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz
Resolving www.vectorbase.org (www.vectorbase.org)... 129.74.255.228
Connecting to www.vectorbase.org (www.vectorbase.org)|129.74.255.228|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz [following]
--2018-05-04 12:02:39--  https://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz
Connecting to www.vectorbase.org (www.vectorbase.org)|129.74.255.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.vectorbase.org/sites/default/files/ftp/downloads/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.2.gff3.gz [following]
--2018-05-04 12:02:40--  https://www.vectorbase.org/sites/default/files/ftp/downloads/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.2.gff3.gz
Reusing existing connection to www.vectorbase.org:443.
HTTP

In [5]:
!rm -f ag.db

In [6]:
try:
    db = gffutils.create_db('gambiae.gff.gz', 'ag.db')
except sqlite3.OperationalError:
    db = gffutils.FeatureDB('ag.db')

In [7]:
print(list(db.featuretypes()))
for feat_type in db.featuretypes():
    print(feat_type, db.count_features_of_type(feat_type))

['CDS', 'RNase_P_RNA', 'SRP_RNA', 'contig', 'exon', 'five_prime_UTR', 'gene', 'mRNA', 'miRNA', 'misc_RNA', 'pseudogene', 'rRNA', 'snRNA', 'snoRNA', 'tRNA', 'tRNA_pseudogene', 'three_prime_UTR']
CDS 62408
RNase_P_RNA 1
SRP_RNA 3
contig 8
exon 66485
five_prime_UTR 10520
gene 13624
mRNA 14697
miRNA 187
misc_RNA 10
pseudogene 5
rRNA 53
snRNA 38
snoRNA 12
tRNA 463
tRNA_pseudogene 9
three_prime_UTR 7281


In [8]:
for contig in db.features_of_type('contig'):
    print(contig)

2L	VectorBase	contig	1	49364325	.	.	.	ID=2L;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
3R	VectorBase	contig	1	53200684	.	.	.	ID=3R;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
UNKN	VectorBase	contig	1	42389979	.	.	.	ID=UNKN;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
X	VectorBase	contig	1	24393108	.	.	.	ID=X;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
Y_unplaced	VectorBase	contig	1	237045	.	.	.	ID=Y_unplaced;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
Mt	VectorBase	contig	1	15363	.	.	.	ID=Mt;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
2R	VectorBase	contig	1	61545105	.	.	.	ID=2R;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
3L	VectorBase	contig	1	41963435	.	.	.	ID=3L;molecule_type=dsDNA;translation_table=1;topology=linear;localization

In [10]:
#XXX Change
num_mRNAs = defaultdict(int)
num_exons = defaultdict(int)
max_exons = 0
max_span = 0
for contig in db.features_of_type('contig'):
    cnt = 0
    for gene in db.region((contig.seqid, contig.start, contig.end), featuretype='gene'):
        cnt += 1
        span = abs(gene.start - gene.end) # strand
        if span > max_span:
            max_span = span
            max_span_gene = gene
        my_mRNAs = list(db.children(gene, featuretype='mRNA'))
        my_exons = list(db.children(gene, featuretype='exon'))
        num_mRNAs[len(my_mRNAs)] += 1
        if len(my_mRNAs) == 0:
            exon_check = [gene]
        else:
            exon_check = my_mRNAs
        for check in exon_check:
            num_exons[len(my_exons)] += 1
            if len(my_exons) > max_exons:
                max_exons = len(my_exons)
                max_exons_gene = gene
    print('contig %s, number of genes %d' % (contig.seqid, cnt))
print('Max number of exons: %s (%d)' % (max_exons_gene.id, max_exons))
print('Max span: %s (%d)' % (max_span_gene.id, max_span))
print(num_mRNAs)
print(num_exons)

contig 2L, number of genes 3105
contig 3R, number of genes 2763
contig UNKN, number of genes 567
contig X, number of genes 1097
contig Y_unplaced, number of genes 0
contig Mt, number of genes 37
contig 2R, number of genes 3834
contig 3L, number of genes 2221
Max number of exons: AGAP010147 (188)
Max span: AGAP006656 (365621)
defaultdict(<class 'int'>, {2: 910, 1: 11595, 3: 211, 4: 74, 0: 781, 11: 3, 5: 27, 8: 4, 12: 1, 7: 5, 6: 9, 13: 1, 10: 1, 20: 1, 9: 1})
defaultdict(<class 'int'>, {8: 481, 2: 3147, 11: 253, 1: 1995, 3: 2379, 19: 82, 21: 79, 31: 22, 23: 70, 10: 316, 6: 869, 4: 1701, 12: 265, 93: 3, 5: 1088, 26: 49, 7: 651, 13: 196, 15: 162, 14: 144, 9: 408, 35: 31, 20: 91, 38: 16, 16: 102, 24: 64, 18: 90, 17: 76, 46: 3, 37: 23, 110: 11, 53: 5, 30: 53, 27: 29, 22: 36, 48: 20, 44: 12, 75: 12, 50: 23, 41: 7, 71: 7, 25: 31, 47: 25, 32: 15, 34: 34, 52: 5, 102: 4, 43: 8, 28: 34, 36: 22, 33: 21, 49: 7, 57: 7, 42: 9, 70: 3, 29: 17, 39: 25, 45: 16, 188: 11, 68: 3, 89: 5, 56: 22, 40: 8, 51: 9